## 初始設定
連結雲端位置、載入套件

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root = '/content/drive/MyDrive/Colab Notebooks/TMBA'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, coint
import statsmodels.api as sm
import yfinance as yf
from datetime import datetime
from tqdm import tqdm

## 載入股價資料

In [ ]:
# 定義開始和結束日期
start_date = datetime(2009, 11, 29)
end_date = datetime(2012, 11, 30)

stock_data = {}

# 論文中有些股票有缺失直所以我就拿掉了

stock_name = ['PEP', 'KO', 'TGT', 'WMT', 'HPQ', 'CVX', 'XOM']

# 載入股價資料

for name in stock_name:
  data = yf.download(name, start=start_date, end=end_date)
  data.reset_index(inplace=True)
  data['log_price'] = np.log(data['Close'])
  data['log_ret'] = data['log_price'] - data['log_price'].iloc[0]
  data.dropna()
  stock_data[name] = data

# 顯示前幾筆資料
print("股價資料:")
for i in stock_name:
  print(stock_data[i].head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
股價資料:
        Date       Open       High        Low      Close  Adj Close   Volume  \
0 2009-11-30  62.639999  62.830002  62.150002  62.220001  41.104282  7252700   
1 2009-12-01  62.680000  63.930000  62.490002  63.869999  42.194313  6887700   
2 2009-12-02  63.500000  63.740002  63.169998  63.650002  42.347336  6195800   
3 2009-12-03  63.580002  63.590000  62.669998  62.759998  41.755203  6406200   
4 2009-12-04  63.180000  64.000000  63.080002  63.849998  42.480392  8907000   

   log_price   log_ret  
0   4

In [ ]:
for i in stock_name:
  print(stock_data[i].shape)

(756, 9)
(756, 9)
(756, 9)
(756, 9)
(756, 9)
(756, 9)
(756, 9)


## ADF 共整合找尋合適的標的

In [ ]:
result = pd.DataFrame()
stock_list_1 = []
stock_list_2 = []

In [ ]:
for i in stock_name:
  for j in stock_name:
    if i != j:
      X = sm.add_constant(stock_data[j]['log_ret'])
      Y = stock_data[i]['log_ret']
      results = sm.OLS(Y, X).fit()
      beta = results.params[1]
      X_t = stock_data[i]['log_price'] - beta * stock_data[j]['log_price'] ## True spread in thesis
      adf_result = adfuller(X_t)
      print(i + " " + j)
      print(adf_result[0])
      if adf_result[0] < -3.5:
          result = result.append({'stock_pair': i + ' and ' + j, 'adf_apir': adf_result[0]}, ignore_index=True)
          stock_list_1.append(i)
          stock_list_2.append(j)
      else:
          pass

PEP KO
-2.329810667448853
PEP TGT
-3.0372973233205043
PEP WMT
-2.630666109819668
PEP HPQ
-2.516980589684447
PEP CVX
-2.7992171427678554
PEP XOM
-2.897110329039811
KO PEP
-1.496435033235266
KO TGT
-1.2779211759091975
KO WMT
-1.7203818971787312
KO HPQ
-1.936776320797971
KO CVX
-2.2753969506743283
KO XOM
-2.400406996005109
TGT PEP
-2.578370944107397
TGT KO
-2.1584166543720427
TGT WMT
-3.4396534593618835
TGT HPQ
-2.7964229763504287
TGT CVX
-1.861157383840838
TGT XOM
-2.169212148365313
WMT PEP
-1.6706335717862029
WMT KO
-1.380175189133393
WMT TGT
-2.642867176430616
WMT HPQ
-2.496291649724969
WMT CVX
-0.9649795286808842
WMT XOM
-1.2720041709608025
HPQ PEP
-1.0260926986934868
HPQ KO
-1.391147021617777
HPQ TGT
-1.6032635627192633
HPQ WMT
-2.237486146284891
HPQ CVX
-1.4524900205397753
HPQ XOM
-1.6131997434639314
CVX PEP
-1.6968482410649903
CVX KO
-2.5210734808535262
CVX TGT
-1.2888460675391307
CVX WMT
-1.7434139500703096
CVX HPQ
-1.966834383988039
CVX XOM
-3.5708279343361835
XOM PEP
-1.58046356

<ipython-input-12-ef5fe8f4b9f9>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'stock_pair': i + ' and ' + j, 'adf_apir': adf_result[0]}, ignore_index=True)


XOM WMT
-1.6965508704508603
XOM HPQ
-1.9861405359556288
XOM CVX
-3.3485407356902392


## 查看符合條件之標的並匯出

In [ ]:
print(stock_list_1)
print(stock_list_2)
print(result)

['CVX']
['XOM']
    stock_pair  adf_apir
0  CVX and XOM -3.570828


In [ ]:
for i in stock_list_1:
  stock_data[i].to_csv(f"{root}/dataset/{i}.csv", index=False)
for i in stock_list_2:
  stock_data[i].to_csv(f"{root}/dataset/{i}.csv", index=False)

In [ ]:
result.to_csv(f"{root}/dataset/stock_pair.csv", index=False)